In [2]:
import tatqa_utils
import pandas as pd

In [3]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [4]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [5]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [6]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [86]:
def gen_code(llm, question, table):    
    #question = question['question']
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    res = llm.invoke(prompt)
    code = res.content.replace('```python','').replace('```','')
    return (prompt, code)

def exec_code(code, table):  
    try: 
        loc = locals()   
        if not "run()" in code:
            exec(code + f"\nr = run({table})\n", globals(), loc)
        else: 
            exec(code + "\nr = run()\n", globals(), loc)
        return loc['r']
    except Exception as e:
            s = '[Error]'+ str(e)
            print(s)
            return (s,'')
#table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
#question = 'What is the change in Fixed Price in 2019 from 2018?'
table = "[['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]"
question = {'question':'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?'}
( prompt, code) = gen_code(llm, question, table)
exec_code(code, table)

(14.065510597302506, '%')

In [87]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    #print (i)
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']: 
            ( prompt, code)  = gen_code(llm, q, table)
            r = exec_code(code, table)
            err =  None
            if r is None:
                (pred_value, pred_unit) = (None, None)
            else:                
                if len(r) == 2:
                    (pred_value, pred_unit) = r                    
                    #print(pred_value, type(pred_value))
                    if isinstance(pred_value, str) and  pred_value.startswith('[Error]'):
                        (pred_value, pred_unit) = (None, None)
                        err,_ = r                                                        
                        
                elif len(r) == 1:
                    pred_value = r                
                else:
                    (pred_value, pred_unit) = (None, None)        
                
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_unit, q, code, prompt, item['table'], err))


0it [00:00, ?it/s]

[Error]invalid literal for int() with base 10: '6,106'
[Error]unterminated string literal (detected at line 3) (<string>, line 3)
[Error]'[' was never closed (<string>, line 2)
[Error]invalid literal for int() with base 10: ''
[Error]'[' was never closed (<string>, line 2)
[Error]could not convert string to float: '1,073.3'
[Error]invalid literal for int() with base 10: '$1625687'
[Error]could not convert string to float: '$1625687'
[Error]invalid literal for int() with base 10: '(7)'
[Error]invalid literal for int() with base 10: '(765)'
[Error]invalid literal for int() with base 10: '—'
[Error]invalid literal for int() with base 10: '$22'
[Error]invalid literal for int() with base 10: '(111)'
[Error]invalid literal for int() with base 10: '38,466'
[Error]invalid literal for int() with base 10: '(71)'
[Error]invalid literal for int() with base 10: '—'
[Error]'[' was never closed (<string>, line 2)
[Error]local variable 'value_2018' referenced before assignment
[Error]unsupported opera

In [76]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=True)
print( pred_em, pred_f1, scale_score)

0.8133903133903134 0.8133903133903134 0.8945868945868946


In [77]:
c = 0
for i, item in devdf.iterrows():
    table = item['table']['table']
    #print (i)
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']: 
            c = c+1
c

702

In [78]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=True)
print( pred_em, pred_f1, scale_score)

0.8133903133903134 0.8133903133903134 0.8945868945868946


In [79]:
import pickle
with open('res_p15_unit.pickle', 'wb') as f:
    pickle.dump(res,f)

In [80]:

for idx,i in enumerate(res):
    if i[-1]is not None:  
        print(idx, i[-2]['uid'], i[3]['uid'],i[-1])


17 789efd09-04a2-45c7-84e5-47d3543695d1 a4f782f1-de3b-47a3-87af-1cb33c112b39 [Error]invalid literal for int() with base 10: '6,106'
21 77d8e381-01d0-4cf9-882e-e1162db2cff2 a983501d-2eec-486d-9661-e520c7c8af5e [Error]unterminated string literal (detected at line 3) (<string>, line 3)
36 386352cf-f7e3-4148-920c-63bc57cf156b 191c3926-7356-4ab8-a8f9-41e7b7c8a492 [Error]'[' was never closed (<string>, line 2)
37 386352cf-f7e3-4148-920c-63bc57cf156b bd2d81eb-46fc-4e62-908d-aebfccf46246 [Error]invalid literal for int() with base 10: ''
38 386352cf-f7e3-4148-920c-63bc57cf156b 010df393-a62e-408d-888a-045a4e435e6a [Error]'[' was never closed (<string>, line 2)
48 0f012382-d9ba-409c-94db-0b9bf50d0746 fa43e8cd-7fea-4738-85ee-f61ae8529f96 [Error]could not convert string to float: '1,073.3'
71 68bafc82-b795-4c7b-9506-e901223c3526 1e20b997-cb3b-4bc5-93e6-b45dbe945fb5 [Error]invalid literal for int() with base 10: '$1625687'
72 68bafc82-b795-4c7b-9506-e901223c3526 3b5bb347-41c7-499a-b044-87b9a04b0833 

In [36]:
item = devdf.iloc[0]
table = item['table']['table']
for q in item['questions']:        
    if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']: 
        print("==========")
        code = gen_code(llm, q, table)
        for l in table:
            print(l)
        print("\n", q['question'],"\n")
        print(code)
        r = exec_code(code, table)
        print(r, q['answer'], q['scale'])

['', '', 'Years Ended September 30,', '']
['', '2019', '2018', '2017']
['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9']
['Other', '44.1', '56.7', '70.8']
['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']

 What is the change in Other in 2019 from 2018? 

def run():
    table = [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$ 1,452.4', '$ 1,146.2', '$ 1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]
    
    value_2019 = float(table[3][1])
    value_2018 = float(table[3][2])
    
    return (value_2019 - value_2018, 'million')
(-12.600000000000001, 'million') -12.6 million
['', '', 'Years Ended September 30,', '']
['', '2019', '2018', '2017']
['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9']
['Other', '44.1', '56.7', '70.8']
['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']

 What is the percentage change in Other in 2019 from 2018? 

def run():
    return (-22.22, 'percent

In [57]:
import re 
def remove_thousand_separators(text):
    # This regular expression matches numbers with commas as thousand separators
    return re.sub(r'(?<=\d),(?=\d{3})', '', text)

remove_thousand_separators('alma 123,432')

'alma 123432'

In [63]:

for idx, item in devdf.iterrows():
    if item['table']['uid'] == '789efd09-04a2-45c7-84e5-47d3543695d1':        
        for q in item['questions']:
            if q['uid'] == 'a4f782f1-de3b-47a3-87af-1cb33c112b39':
                table = item['table']['table']
                table = [remove_thousand_separators(cell) for row in table for cell in row]
                if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']: 
                    print("==========")
                    code = gen_code(llm, q, table)
                    for l in table:
                        print(l)
                    print("\n", q['question'],"\n")
                    print(code)
                    r = exec_code(code, table)
                    print(r, q['answer'], q['scale'])


December 31,


2019
2018
Trade accounts receivable, net, noncurrent (Note 2)
$26496
$15948
Equity method investments (Note 1)
9254
9702
Net deferred tax assets, noncurrent (Note 20)
6774
5797
Rent and other deposits
6106
5687
Value added tax receivables, net, noncurrent
592
519
Other
6723
5711

$55945
$43364

 What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? 

def run():
    return (14.07, 'percent')
(14.07, 'percent') 14.07 percent


In [60]:
table

[['', 'December 31,', ''],
 ['', '2019', '2018'],
 ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'],
 ['Equity method investments (Note 1)', '9,254', '9,702'],
 ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'],
 ['Rent and other deposits', '6,106', '5,687'],
 ['Value added tax receivables, net, noncurrent', '592', '519'],
 ['Other', '6,723', '5,711'],
 ['', '$55,945', '$43,364']]

In [88]:
res[0]


({'answer_type': 'arithmetic', 'answer': -12.6, 'scale': 'million'},
 -12.600000000000001,
 'million',
 {'uid': 'eb787966-fa02-401f-bfaf-ccabf3828b23',
  'order': 5,
  'question': 'What is the change in Other in 2019 from 2018?',
  'answer': -12.6,
  'derivation': '44.1-56.7',
  'answer_type': 'arithmetic',
  'answer_from': 'table-text',
  'rel_paragraphs': ['2'],
  'req_comparison': False,
  'scale': 'million'},
 "def run():\n    table = [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$ 1,452.4', '$ 1,146.2', '$ 1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]\n    \n    value_2019 = float(table[3][1])\n    value_2018 = float(table[3][2])\n    \n    return (value_2019 - value_2018, 'million')",
 "Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. Th